### División de texto

In [ ]:
import pandas as pd
import spacy
nlp = spacy.load("es_core_news_md")

In [ ]:
spacy.info()

In [ ]:
doc = nlp("La gata de Juan es blanca.")

In [ ]:
[t for t in doc]

In [ ]:
[t.text for t in doc]

In [ ]:
[t.lower_ for t in doc]

In [ ]:
doc = nlp("la vaca come hierba. El perro come longanizas.")

In [ ]:
[s for s in doc.sents]

In [ ]:
[s.text for s in doc.sents]

### Limpieza de acentos

In [ ]:
import unicodedata

data = 'Sómě Áccěntěd tëxt'
normal = unicodedata.normalize('NFKD', data).encode('ASCII', 'ignore')
print(normal)

In [ ]:
def remove_accents(text):
    new_text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    return new_text

In [ ]:
remove_accents(data)

In [ ]:
remove_accents(data) + ' adiós'

In [ ]:
normal + ' adiós'

In [ ]:
from gensim.utils import deaccent
#https://radimrehurek.com/gensim/utils.html#gensim.utils.deaccent

In [ ]:
deaccent(data)

In [ ]:
help(deaccent)

### Limpieza de caracteres especiales

In [ ]:
import re, string

def remove_special_characters(text):
    pat = '[{}]'.format(re.escape(string.punctuation))
    return re.sub(pat, '', text)
 
texto = "007 Not sure@,of course if this % was #fun! 558923 What do# you think** of it.? $500USD!"
remove_special_characters(texto)

In [ ]:
string.punctuation

In [ ]:
'[{}]'.format(re.escape(string.punctuation))

In [ ]:
len([t for t in nlp(texto)])

In [ ]:
len([t for t in nlp(remove_special_characters(texto))])

### Expandir contracciones
hay que instalar la librería https://github.com/kootenpv/contractions con ```pip install contractions```

In [ ]:
import contractions
contractions.fix("you're happy now, aren't you?")

In [ ]:
nlp_en = spacy.load("en_core_web_md")

In [ ]:
doc = nlp_en("you're happy now, aren't you?")
[t for t in doc]

### Stop-words

In [ ]:
from spacy.lang.en.stop_words import STOP_WORDS

In [ ]:
len(STOP_WORDS)

In [ ]:
print(STOP_WORDS)

In [ ]:
import spacy    
nlp = spacy.load('es_core_news_sm')
print(len(nlp.Defaults.stop_words))
print(nlp.Defaults.stop_words)

Cuidado con las negaciones

In [ ]:
'no' in nlp.Defaults.stop_words

In [ ]:
'tí' in nlp.Defaults.stop_words

Los stop-words tienen el atributo `is_stop` a `TRUE`

In [ ]:
doc = nlp("Hoy no me apetece ir a dar clase")
datos = map(lambda t: {'token': t.orth_,
                       'STOP': t.is_stop},
                        doc)

pd.DataFrame(datos)

In [ ]:
'todavia' in nlp.Defaults.stop_words

In [ ]:
'usáis' in nlp.Defaults.stop_words

In [ ]:
#en cambio la palabra incorrecta
'usais' in nlp.Defaults.stop_words

In [ ]:
#podemos añadir o quitar palabras de la lista

#añadir
nlp.Defaults.stop_words.add("my_new_stopword")
nlp.Defaults.stop_words |= {"my_new_stopword1","my_new_stopword2"}

#quitar
nlp.Defaults.stop_words.remove("usais")
nlp.Defaults.stop_words -= {"tuya", "mia"}

In [ ]:
"usais" in nlp.Defaults.stop_words

In [ ]:
from gensim.parsing.preprocessing import remove_stopwords
import gensim
gensim_stopwords = gensim.parsing.preprocessing.STOPWORDS
text = f"The first time I saw Catherine she was wearing a vivid crimson dress and was nervously " \
       f"leafing through a magazine in my waiting room."
print(f"Original Text : {text}")
print(f"Text without stopwords : {remove_stopwords(text.lower())}")
print(f"Total count of stopwords in Gensim is {len(list(gensim_stopwords))}")

### Corrección ortográfica
Librería `spellchecker`. Instalamos con
```pip install pyspellchecker```

In [ ]:
from spellchecker import SpellChecker

spell = SpellChecker(language='es')  # Spanish dictionary
print(f"Hay {spell.word_frequency._unique_words} palabras en el diccionario")

In [ ]:
spell.correction('mañnaa')

In [ ]:
spell.candidates('mañnaa')

In [ ]:
#si una palabra está en el diccionario devuelve su frecuencia relativa:
spell['mañana']  #equivale a spell.word_frequency['mañana']

In [ ]:
spell['mañnaa']

Si una palabra está en el diccionario, aunque que la frecuencia sea baja no la corregirá.

In [ ]:
spell['mañna']

In [ ]:
spell.correction('mañna')

### Lematizado

In [ ]:
import spacy
nlp = spacy.load("es_core_news_sm")
doc = nlp("el gato es blanco")
[t.lemma_ for t in doc]

In [ ]:
doc = nlp("la vaca come hierba. Los perros comen longanizas.")
[s.lemma_ for s in doc.sents]

In [ ]:
doc = nlp("la salida se ha bloqueado. La salida está bloqueada.")
[(t.lemma_, t.pos_) for t in doc]

### Funciones de normalización

In [ ]:
texto = """@Graffitera23 qué hermoso!,es bueno desviar la
mirada al cielo y a las #nubes de vez en cuando, 
abajo está jodido. Preciosa foto,mil abrazos """

In [ ]:
# en spacy
import re
import spacy
nlp=spacy.load('es_core_news_sm')
               
def normalize_document(doc):
   # separamos en tokens
    tokens = nlp(doc)
    # quitamos puntuación/espacios y stopwords
    filtered_tokens = [t.lower_ for t in tokens if not t.is_stop and not t.is_punct]
    # juntamos de nuevo en una cadena
    doc = ' '.join(filtered_tokens)

    return doc

In [ ]:
normalize_document(texto)

Con esta función no se eliminan los signos de puntuación que no forman un token de manera independiente, debemos hacerlo con un patrón regular

In [ ]:
import string

pat  = '[{}]'.format(re.escape(string.punctuation))

def normalize_document_remove_punct(doc):
   # separamos en tokens
    tokens = nlp(doc)
    # quitamos puntuación/espacios y stopwords
    filtered_tokens = [re.sub(pat, ' ', t.lower_) 
      for t in tokens
         if not t.is_stop
         and not t.is_punct
         and not t.is_space]
    # juntamos de nuevo en una cadena
    doc = ' '.join(filtered_tokens)

    return doc

In [ ]:
normalize_document_remove_punct(texto)

Con la librería `gensim` tenemos acceso a diversas funciones de normalización que convierten una cadena de texto en un listado de tokens normalizados

In [ ]:
from gensim.utils import simple_preprocess
#https://radimrehurek.com/gensim/utils.html#gensim.utils.simple_preprocess

help(simple_preprocess)

In [ ]:
simple_preprocess(texto, deacc=True, min_len=0)

In [ ]:
from gensim.utils import tokenize
#https://radimrehurek.com/gensim/utils.html#gensim.utils.tokenize

help(tokenize)

In [ ]:
tokenize(texto)

In [ ]:
list(tokenize(texto, deacc=True, lowercase=True))

In [ ]:
from gensim.parsing.preprocessing import preprocess_string
#https://radimrehurek.com/gensim/parsing/preprocessing.html#gensim.parsing.preprocessing.preprocess_string
help(preprocess_string)

In [ ]:
preprocess_string(texto) #elimina stop words y deja raíz de las palabras

In [ ]:
preprocess_string("<i>Hel 9lo</i> <b>Wo9 rld</b>! Th3     weather_is really g00d today, isn't it?")

In [ ]:
preprocess_string("Transformer is behind the recent NLP developments, including Google’s BERT")

In [ ]:
from gensim.parsing.preprocessing import *
preprocess_string("Transformer is behind the recent NLP developments, including Google’s BERT", [remove_stopwords])

In [ ]:
preprocess_string("Transformer is behind the recent NLP developments, including Google’s BERT", [remove_stopwords, stem_text])

También la librería `textaCy` proporciona diversas funciones de preprocesado (https://textacy.readthedocs.io/en/latest/api_reference/preprocessing.html).  
La instalamos con  
`conda install -c conda-forge textacy`

In [ ]:
from textacy import preprocessing
preproc = preprocessing.make_pipeline(
     preprocessing.replace.hashtags,
     preprocessing.replace.user_handles,
     preprocessing.replace.emojis,
     preprocessing.remove.accents
 )
preproc("Según @valencia, hoy comienzan las #fallas con la primera mascletà 🔥🎇")
